## データ収集

In [ ]:
# !pip install lxml
# !pip install html5lib
# !pip install bs4
# !pip install imblearn

In [ ]:
import pandas as pd

In [ ]:
# サンプル確認
url = 'https://db.netkeiba.com/race/202009030811/'

print(type(pd.read_html(url)[0]))
# -> df

pd.read_html(url)[0]
# -> 	着順	枠番	馬番	馬名	性齢	斤量	騎手	タイム	着差	単勝	人気	馬体重	調教師

In [ ]:
import time

def scrape_race_results(race_id_list: list)-> dict:
    """
    netkeiba.comのレースIDのリストを渡して、それらをまとめて{'レースID', 結果のDataFrame}という形式の辞書型に格納する
    race_results['201901010101']
    -> df 着順	枠番	馬番	馬名	性齢	斤量	騎手	タイム	着差	単勝	人気	馬体重	調教師
    """
    race_results_dict = {}
    for race_id in race_id_list:
        try:
            url = 'https://db.netkeiba.com/race/' + race_id
            race_results_dict[race_id] = pd.read_html(url)[0]
            time.sleep(0.1)
        except IndexError:
            continue
        except:
            break
    return race_results_dict

In [ ]:
# race id を生成する（規則的に生成できる）
# todo: 2019年に絞っている
race_id_list = []

for place in range(1,11):
    for kai in range(1,6):
        for day in range(1,9):
            for r in range(1,13):
                race_id = '2019' + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)
# -> ['201901010101', '201901010102', ,,, ]

In [ ]:
# todo: listを絞っている
race_results_dict = scrape_race_results(race_id_list[0:50])

In [ ]:
# 表として見やすいように、dfのindexにrace idを入れる
for key in race_results_dict.keys():
    race_results_dict[key].index = [key]*len(race_results_dict[key])

# 各レース結果のdfを1つに結合する
race_results_df = pd.concat((race_results_dict[key] for key in race_results_dict.keys()),sort=False)

In [ ]:
race_results_df.to_pickle('../../../data/raw/race_results_df.pickle')

## 前処理

In [ ]:
import pandas as pd

In [ ]:
race_results_df = pd.read_pickle('../../../data/raw/race_results_df.pickle')

In [ ]:
def preprocess_netkeiba_past(race_results_df):
    df = race_results_df.copy()

    # データ整形
    df = df[~(df['着順'].astype(str).str.contains('\D'))]
    df['着順'] = df['着順'].astype(int)
    df['性'] = df['性齢'].map(lambda x:str(x)[0])
    df['年齢'] = df['性齢'].map(lambda x:str(x)[1:]).astype(int)
    df['体重'] = df['馬体重'].str.split('(',expand = True)[0].astype(int)
    df['体重変化'] = df['馬体重'].str.split('(',expand = True)[1].str[:-1].astype(int)
    df['単勝'] = df['単勝'].astype(float)
    
    df.drop(['タイム','着差','調教師','性齢','馬体重'],axis = 1,inplace = True)

    # 4位より下はまとめる
    clip_rank = lambda x: x if x < 4 else 4
    df['rank'] = df['着順'].map(clip_rank)

    # test['馬名'].value_counts()などでカウントし、数が多そうなのは落とした後、ダミー変数化
    df.drop(['着順','馬名'], axis = 1,inplace = True)
    df = pd.get_dummies(df)
    
    return df

In [ ]:
race_results_df_processed = preprocess_netkeiba_past(race_results_df)

In [ ]:
race_results_df_processed.to_pickle('../../../data/processed/race_results_df_processed.pickle')

## 学習

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
race_results_df_processed = pd.read_pickle('../../../data/processed/race_results_df_processed.pickle')

In [ ]:
# 説明変数の取得
X = race_results_df_processed.drop(['rank'],axis=1)
# 目的変数の取得
y = race_results_df_processed['rank']

# train と test に分離
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=0)

In [ ]:
# 下記で学習もできるが、今回はunder samplingを追加するのでコメントアウトする
# # 学習
# model = LogisticRegression()
# model.fit(X_train,y_train)
# print('train score: ' + str(model.score(X_train,y_train)))
# print('test score: ' + str(model.score(X_test,y_test)))

# # テストデータでの予測結果を取得し、出力する
# y_pred = model.predict(X_test)
# print(pd.DataFrame({'pred':y_pred,'actual':y_test}))

In [ ]:
# ランダムアンダーサンプリング
from imblearn.under_sampling import RandomUnderSampler

cnt_rank_1 = y_train.value_counts()[1]
cnt_rank_2 = y_train.value_counts()[2]
cnt_rank_3 = y_train.value_counts()[3]

rus = RandomUnderSampler(sampling_strategy={1:cnt_rank_1,2:cnt_rank_2,3:cnt_rank_3,4:cnt_rank_1},random_state=71)

X_train_rus,y_train_rus = rus.fit_sample(X_train,y_train)

In [ ]:
# 学習
model = LogisticRegression()
model.fit(X_train_rus,y_train_rus)
print('train score: ' + str(model.score(X_train,y_train)))
print('test score: ' + str(model.score(X_test,y_test)))

In [ ]:
# テストデータでの予測結果を取得し、出力する
y_pred = model.predict(X_test)
print(pd.DataFrame({'pred':y_pred,'actual':y_test}))